In [1]:
import os
os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=gpu,floatX=float32"
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from copy import deepcopy
import time
import matplotlib.pyplot as plt
% matplotlib inline

In [2]:
# Grab all data from parquet file
dataset = pd.read_parquet('Metal_all_20180116.snappy.parquet')

# Grab only the zinc binding sequences
zinc_dataset = dataset.loc[dataset['ligandId'] == 'ZN']

In [3]:
# Boolean Column to indicate which row contains 'X' and 'U'.
xub_flag = list()
# This will be used later to get rid of outliers
seq_len = list()   

# Fill in 'xub_flag' and 'seq_len'
for row in zinc_dataset.itertuples() :
    if ('X' in row[5]) or ('U' in row[5]) or ('B' in row[5]) : xub_flag.append(1)
    else : xub_flag.append(0)
    seq_len.append( len(row[5]) )
    
# Append the columns
xub_col = pd.DataFrame({'xub_col' : xub_flag})
length_col = pd.DataFrame({'seq_length' : seq_len})
# Reset Index
xub_col = xub_col.reset_index(drop=True)
length_col = length_col.reset_index(drop=True)
zinc_dataset = zinc_dataset.reset_index(drop=True)
# Combine the columns
zinc_dataset = zinc_dataset.join(xub_col)
zinc_dataset = zinc_dataset.join(length_col)

# Exclude sequences containing 'X' and 'U' and 'B'
zinc_dataset = zinc_dataset[zinc_dataset.xub_col != 1]
zinc_dataset = zinc_dataset.drop('xub_col', axis=1)
zinc_dataset

,structureChainId,ligandId,fingerprint,groupNumber,sequence,interactingChains,clusterNumber30,clusterNumber40,clusterNumber50,clusterNumber70,clusterNumber90,clusterNumber95,clusterNumber100,seq_length
0,1AJD.B,ZN,"[50, 101, 368, 369]","[51, 102, 369, 370]",TPEMPVLENRAAQGDITAPGGARRLTGDQTAALRDSLSDKPAKNII...,1,584.0,592.0,555.0,515.0,485.0,444.0,8044.0,449
1,1AJD.B,ZN,"[326, 330, 411]","[327, 331, 412]",TPEMPVLENRAAQGDITAPGGARRLTGDQTAALRDSLSDKPAKNII...,1,584.0,592.0,555.0,515.0,485.0,444.0,8044.0,449
2,1ALN.A,ZN,"[101, 128, 131]","[102, 129, 132]",MHPRFQTAFAQLADNLQSALEPILADKYFPALLTGEQVSSLKSATG...,1,4526.0,4989.0,13948.0,15950.0,17149.0,17499.0,19782.0,294
3,1CG2.A,ZN,"[89, 118, 177]","[112, 141, 200]",ALAQKRDNVLFQAATDEQPAVIKTLEKLVNIETGTGDAEGIAAAGN...,1,10402.0,12008.0,13323.0,15154.0,16224.0,16540.0,18316.0,393
4,1CG2.A,ZN,"[118, 153, 362]","[141, 176, 385]",ALAQKRDNVLFQAATDEQPAVIKTLEKLVNIETGTGDAEGIAAAGN...,1,10402.0,12008.0,13323.0,15154.0,16224.0,16540.0,18316.0,393
5,1CK1.A,ZN,"[82, 117, 121]","[83, 118, 122]",ESQPDPMPDDLHKSSEFTGTMGNMKYLYDDHYVSATKVKSVDKFLA...,1,329.0,486.0,498.0,1420.0,1401.0,1376.0,27622.0,239
6,1E31.B,ZN,"[56, 59, 76, 83]","[57, 60, 77, 84]",MGAPTLPPAWQPFLKDHRISTFKNWPFLEGCACTPERMAEAGFIHC...,1,1654.0,1691.0,1674.0,1634.0,1678.0,1642.0,2605.0,142
7,1IAG.A,ZN,"[141, 145, 151]","[142, 146, 152]",ENLPQRYIELVVVADRRVFMKYNSDLNIIRTRVHEIVNIINKFYRS...,1,1240.0,1730.0,2154.0,6944.0,18434.0,18919.0,62207.0,202
8,1LR5.C,ZN,"[56, 58, 62, 105]","[57, 59, 63, 106]",SCVRDNSLVRDISQMPQSSYGIEGLSHITVAGALNHGMKEVEVWLQ...,1,6660.0,7552.0,8231.0,9103.0,9534.0,9649.0,9489.0,163
9,1TO5.C,ZN,"[64, 72, 81, 84]","[62, 70, 79, 82]",GSNMKAVCVMTGTAGVKGVVKFTQETDNGPVHVHAEFSGLKAGKHG...,1,66.0,70.0,55.0,7056.0,9563.0,9678.0,9527.0,156


In [4]:
# Normalize length
min_max_scaler = preprocessing.MinMaxScaler()
length_scaled = min_max_scaler.fit_transform(length_col)
length_scaled = length_scaled.ravel()
#length_normalized = pd.DataFrame(length_scaled).values.tolist()
length_normalized = pd.DataFrame({'length_norm' : length_scaled})

# Combine this back to zinc_dataset
zinc_dataset = zinc_dataset.join( length_normalized )

In [5]:
mean = np.mean(zinc_dataset.iloc[:,-1])
std = np.std(zinc_dataset.iloc[:,-1])
upper_bound = mean - 1*std
lower_bound = mean + 1*std
print('mean : ', mean)
print('std : ', std)
print('lower bound : ', mean - 1*std)
print('upper bound : ', mean + 1*std)

mean :  0.1639523424866267
std :  0.11698831776993822
lower bound :  0.04696402471668849
upper bound :  0.28094066025656494


In [6]:
rm_ind = list()
index = 0

# store indices to remove later
for row in zinc_dataset.itertuples() :
    if (row[-1] < upper_bound) or (lower_bound < row[-1]) :
        rm_ind.append(index)
    index += 1

# reverse sort the index
rm_ind.sort(reverse=True)

# remove rows
for index in rm_ind : 
    zinc_dataset = zinc_dataset.drop(zinc_dataset.index[index])

# remove unnecessary column
zinc_dataset = zinc_dataset.drop('length_norm', axis=1)

In [7]:
zinc_dataset

,structureChainId,ligandId,fingerprint,groupNumber,sequence,interactingChains,clusterNumber30,clusterNumber40,clusterNumber50,clusterNumber70,clusterNumber90,clusterNumber95,clusterNumber100,seq_length
0,1AJD.B,ZN,"[50, 101, 368, 369]","[51, 102, 369, 370]",TPEMPVLENRAAQGDITAPGGARRLTGDQTAALRDSLSDKPAKNII...,1,584.0,592.0,555.0,515.0,485.0,444.0,8044.0,449
1,1AJD.B,ZN,"[326, 330, 411]","[327, 331, 412]",TPEMPVLENRAAQGDITAPGGARRLTGDQTAALRDSLSDKPAKNII...,1,584.0,592.0,555.0,515.0,485.0,444.0,8044.0,449
2,1ALN.A,ZN,"[101, 128, 131]","[102, 129, 132]",MHPRFQTAFAQLADNLQSALEPILADKYFPALLTGEQVSSLKSATG...,1,4526.0,4989.0,13948.0,15950.0,17149.0,17499.0,19782.0,294
3,1CG2.A,ZN,"[89, 118, 177]","[112, 141, 200]",ALAQKRDNVLFQAATDEQPAVIKTLEKLVNIETGTGDAEGIAAAGN...,1,10402.0,12008.0,13323.0,15154.0,16224.0,16540.0,18316.0,393
4,1CG2.A,ZN,"[118, 153, 362]","[141, 176, 385]",ALAQKRDNVLFQAATDEQPAVIKTLEKLVNIETGTGDAEGIAAAGN...,1,10402.0,12008.0,13323.0,15154.0,16224.0,16540.0,18316.0,393
5,1CK1.A,ZN,"[82, 117, 121]","[83, 118, 122]",ESQPDPMPDDLHKSSEFTGTMGNMKYLYDDHYVSATKVKSVDKFLA...,1,329.0,486.0,498.0,1420.0,1401.0,1376.0,27622.0,239
6,1E31.B,ZN,"[56, 59, 76, 83]","[57, 60, 77, 84]",MGAPTLPPAWQPFLKDHRISTFKNWPFLEGCACTPERMAEAGFIHC...,1,1654.0,1691.0,1674.0,1634.0,1678.0,1642.0,2605.0,142
7,1IAG.A,ZN,"[141, 145, 151]","[142, 146, 152]",ENLPQRYIELVVVADRRVFMKYNSDLNIIRTRVHEIVNIINKFYRS...,1,1240.0,1730.0,2154.0,6944.0,18434.0,18919.0,62207.0,202
8,1LR5.C,ZN,"[56, 58, 62, 105]","[57, 59, 63, 106]",SCVRDNSLVRDISQMPQSSYGIEGLSHITVAGALNHGMKEVEVWLQ...,1,6660.0,7552.0,8231.0,9103.0,9534.0,9649.0,9489.0,163
9,1TO5.C,ZN,"[64, 72, 81, 84]","[62, 70, 79, 82]",GSNMKAVCVMTGTAGVKGVVKFTQETDNGPVHVHAEFSGLKAGKHG...,1,66.0,70.0,55.0,7056.0,9563.0,9678.0,9527.0,156


In [8]:
maxlength = max(zinc_dataset['seq_length'])
maxlength

506

## Split Data into Training and Testing

In [9]:
# Split data into train:test (8:2)
train, test = train_test_split(zinc_dataset, test_size=0.33)

### Convert Data to Autocorrelation 

In [10]:
# Read in reference table
ref_table = pd.read_excel('sample_AAproperty.xlsx')
property_names = ref_table['property'].values
#ref_table = ref_table.drop('property', 1)
ref_table

,property,A,C,D,E,F,G,H,I,K,...,M,N,P,Q,R,S,T,V,W,Y
0,K_0,-25.50,-32.82,-33.12,-36.17,-34.54,-27.00,-31.84,-31.78,-32.40,...,-31.18,-30.90,-23.35,-32.60,-26.62,-29.88,-31.23,-39.62,-30.24,-35.01
1,H_t,0.87,1.52,0.66,0.67,2.87,0.10,0.87,3.15,1.64,...,1.67,0.09,2.77,0.00,0.85,0.07,0.07,1.87,3.77,2.67
2,H_p,13.05,14.30,11.10,11.41,13.89,12.20,12.42,15.34,11.01,...,13.62,11.72,11.06,11.78,12.40,11.68,12.12,14.73,13.96,13.57
3,P,0.00,1.48,49.70,49.90,0.35,0.00,51.60,0.10,49.50,...,1.43,3.38,1.58,3.53,52.00,1.67,1.66,0.13,2.10,1.61
4,pH_i,6.00,5.05,2.77,5.22,5.48,5.97,7.59,6.02,9.74,...,5.74,5.41,6.30,5.65,10.76,5.68,5.66,5.96,5.89,5.66
5,pK',2.34,1.65,2.01,2.19,1.89,2.34,1.82,1.36,2.18,...,2.28,2.02,1.99,2.17,1.81,2.21,2.10,2.32,2.38,2.20
6,M_w,89.00,121.00,133.00,147.00,165.00,75.00,155.00,131.00,146.00,...,149.00,132.00,115.00,146.00,174.00,105.00,119.00,117.00,204.00,181.00
7,P_1,11.50,13.46,11.68,13.57,19.80,3.40,13.67,21.40,15.71,...,16.25,12.82,17.43,14.45,14.28,9.47,15.77,21.57,21.61,18.03
8,R_f,9.90,2.80,2.80,3.20,18.80,5.60,8.20,17.10,3.50,...,14.70,5.40,14.80,9.00,4.60,6.90,9.50,14.30,17.00,15.00
9,m,14.34,35.77,12.00,17.26,29.40,0.00,31.81,19.06,21.29,...,21.64,13.28,10.93,17.56,26.66,6.53,11.01,13.92,42.53,31.55


In [11]:
def aasa(l, p, k, seq) :
    '''
    ' Parameter :
    '    l - spatial lag (int value)
    '    p - reference dataframe that holds all property values
    '    k - type of property
    '    seq - protein sequence
    '''
    summation = 0.0
    nonzeros = len(seq) - l
    
    for i in range( len(seq)-l ) :
        # Get indices
        fi = i
        li = i + l
        # Multiply them out and sum them
        p = ref_table.loc[k, seq[fi]] * ref_table.loc[k, seq[li]]
        summation += p
    return summation/nonzeros

In [12]:
def create_blocks(lag, ref_table, sequence) : 
    df_dict = {}
    counter = 0
    lag_names = list()

    # Automate lag names
    [lag_names.append("lag%i" %(i)) for i in range(1, lag+1)]
    
    # Actual Algorithm to fill the Dataframe with AASA
    for each_prop in range(len(ref_table)) :
        prop = list()
        for sp_lag in range(1, lag+1) :
            prop.append( aasa(sp_lag, ref_table, each_prop, sequence) )
        df_dict[property_names[each_prop]] = prop
        
    # Return DataFrame    
    return pd.DataFrame(df_dict, index=lag_names)

In [13]:
# SAMPLE
seq = 'MGAPTLPPAWQPFLKDHRISTFKNWPFLEGCACTPERMAEAGFIHCPTENEPDLAQCFFCFKELEGWEPDDDPIEEHKKHSSGCAFLSVKKQFEELTLGEFLKLDRERAKNKIAKETNNKKKEFEETAKKVRRAIEQLAAMD'
sample_df = create_blocks(5, ref_table, seq)
sample_df

,(neg)TdeltaS,E_sm,H_nc,H_p,H_t,K_0,M_w,P,P_1,R_f,delta_G,delta_H,m,pH_i,pK'
lag1,0.550977,1.459748,0.097128,155.208840,1.785567,969.827710,17721.184397,448.987319,227.516708,82.079220,0.055740,0.935561,341.026118,39.979405,4.388947
lag2,0.635674,1.457256,-0.054280,154.964307,1.831116,971.727698,17774.664286,380.862461,228.595429,79.368000,0.075129,1.187316,350.958790,40.041531,4.387849
lag3,0.631906,1.454646,0.260908,155.324941,1.955691,973.192718,17851.395683,423.623553,229.757533,87.402446,0.092671,1.312854,344.521318,39.780342,4.371623
lag4,0.077811,1.449711,0.031217,155.115914,1.717159,976.744203,17853.768116,402.312725,228.861961,81.529275,-0.019054,0.101946,344.905433,39.644044,4.366106
lag5,0.000334,1.449496,-0.076150,154.921608,1.797334,979.547492,17891.729927,391.608160,227.876121,82.146204,-0.022812,-0.107474,341.596225,39.776018,4.359675


In [14]:
rawX_train = deepcopy(train['sequence'])
rawX_test = deepcopy(test['sequence'])
rawy_train = deepcopy(train['fingerprint'])
rawy_test = deepcopy(test['fingerprint'])

In [15]:
# Convert training input values to matrix of AASA
X_train = [0] * len(rawX_train)
nlags = 12
index = 0
for seq in rawX_train :
    aasa_block = create_blocks(nlags, ref_table, seq).as_matrix()
    scaled_block = preprocessing.scale(np.array(aasa_block))
    X_train[index] = scaled_block
    index = index + 1
    if index % 50 == 0 : 
        print(index)

50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
3850
3900
3950
4000
4050
4100
4150
4200
4250
4300
4350
4400
4450
4500
4550
4600
4650
4700
4750
4800
4850
4900
4950
5000
5050
5100
5150
5200
5250
5300
5350
5400
5450
5500
5550
5600
5650
5700
5750
5800
5850
5900
5950
6000
6050
6100
6150
6200
6250
6300
6350
6400
6450
6500
6550
6600
6650
6700
6750
6800
6850
6900
6950
7000
7050
7100
7150
7200
7250
7300
7350
7400
7450
7500
7550
7600
7650
7700
7750
7800
7850
7900
7950
8000
8050
8100
8150
8200
8250
8300
8350
8400
8450
8500
8550
8600
8650
8700
8750
8800
8850
8900
8950
9000
9050
9100
9150
9200
9250
9300
9350
9400
9450
9500
9550
9600
9650
9700
9750
9800
9850
9900
9950
10000
10050
10100
10150
1

In [16]:
# Convert test input values to matrix of AASA
X_test = [0] * len(rawX_test)
nlags = 12
index = 0
for seq in rawX_test :
    aasa_block = create_blocks(nlags, ref_table, seq).as_matrix()
    scaled_block = preprocessing.scale(np.array(aasa_block))
    X_test[index] = scaled_block
    index = index + 1
    if index % 50 == 0 : 
        print(index)

50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
3850
3900
3950
4000
4050
4100
4150
4200
4250
4300
4350
4400
4450
4500
4550
4600
4650
4700
4750
4800
4850
4900
4950
5000
5050
5100
5150
5200
5250
5300
5350
5400
5450
5500
5550
5600
5650
5700
5750
5800
5850
5900
5950
6000


In [17]:
y_train = list()

# Use fingerprint indices to label the location of where metal binded.
for fingerprints in rawy_train :
    label = [0] * maxlength
    for fingerprint in fingerprints : 
        label[fingerprint] = 1
    y_train.append(label)

In [18]:
y_test = list()

# Use fingerprint indices to label the location of where metal binded.
for fingerprints in rawy_test :
    label = [0] * maxlength
    for fingerprint in fingerprints : 
        label[fingerprint] = 1
    y_test.append(label)

### Normalize (Min-Max Normalization) X_train/X_test by columns
We do Min-Max since our data composed of signed values.

In [19]:
scaler = MinMaxScaler()

In [20]:
# Min-Max Normalization on Train
for i in range( len(X_train) ) :
    temp = pd.DataFrame( X_train[i] )
    scaler.fit(temp)
    X_train[i] = scaler.transform(temp)

In [21]:
# Min-Max Normalization on Test
for i in range( len(X_test) ) :
    temp = pd.DataFrame( X_test[i] )
    scaler.fit(temp)
    X_test[i] = scaler.transform(temp)

## Build CNN

In [22]:
# Importing all the necessary libraries to build CNN
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import AveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Dense
from keras import optimizers
from keras.callbacks import ReduceLROnPlateau

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [23]:
batch_size = 128
epochs = 20
feature_num = 15
X_train_size = len(X_train)
X_test_size = len(X_test)
width = X_train[0].shape[1]
height = X_train[0].shape[0]

In [32]:
'''
# THIS MODEL HAD F1 : 67.185%

classifier = Sequential()
# This will create 32 Feature Maps. Input shape is referring to single sample.
classifier.add(Conv2D(64, (3,3), input_shape=(width, height, 1), activation="relu", padding='same', kernel_initializer='he_normal'))
classifier.add(Conv2D(64, (3,3), input_shape=(width, height, 1), activation="relu", padding='same', kernel_initializer='he_normal'))
#classifier.add(MaxPooling2D(pool_size=(2,2), data_format="channels_first")) # (vertical, horizontal)
classifier.add(MaxPooling2D(pool_size=(2,1))) # (vertical, horizontal)
#classifier.add(Dropout(0.1))
# Don't need 'input_shape' because it's already given from previous step.
classifier.add(Conv2D(128, (3,3), activation="relu", padding='same', kernel_initializer='he_normal'))
#classifier.add(MaxPooling2D(pool_size=(2,2), data_format="channels_first")) # (vertical, horizontal)
#classifier.add(MaxPooling2D(pool_size=(2,1))) # (vertical, horizontal)
#classifier.add(Dropout(0.1))

classifier.add(Flatten())
# Choose units to be not too big but not too small. Common practice is to pick power of 2.
classifier.add(Dense(units=512, activation="relu"))
classifier.add( BatchNormalization() )
#classifier.add( Dropout(0.1) )
classifier.add( Dense(maxlength, activation='softmax') )
'''


'''
# f1 = 0.68726 with Adam Optimizer and f1 = 0.6905 with SGD and f1 = 0.6939 with SGD and BatchNorm in Dense Layer.
classifier = Sequential()
# This will create 32 Feature Maps. Input shape is referring to single sample.
classifier.add(Conv2D(64, (3,3), input_shape=(width, height, 1), activation="relu", padding='same', kernel_initializer='he_normal'))
classifier.add(Conv2D(64, (3,3), input_shape=(width, height, 1), activation="relu", padding='same', kernel_initializer='he_normal'))
classifier.add( BatchNormalization() )
#classifier.add(MaxPooling2D(pool_size=(2,2), data_format="channels_first")) # (vertical, horizontal)
classifier.add(MaxPooling2D(pool_size=(2,1))) # (vertical, horizontal)
#classifier.add(Dropout(0.1))
# Don't need 'input_shape' because it's already given from previous step.
classifier.add(Conv2D(128, (3,3), activation="relu", padding='same', kernel_initializer='he_normal'))
#classifier.add(Conv2D(128, (3,3), activation="relu", padding='same', kernel_initializer='he_normal'))
classifier.add( BatchNormalization() )
#classifier.add(MaxPooling2D(pool_size=(2,2), data_format="channels_first")) # (vertical, horizontal)
#classifier.add(MaxPooling2D(pool_size=(2,1))) # (vertical, horizontal)
#classifier.add(Dropout(0.1))

classifier.add(Flatten())
# Choose units to be not too big but not too small. Common practice is to pick power of 2.
classifier.add(Dense(units=512, activation="relu"))
classifier.add( BatchNormalization() )
#classifier.add( Dropout(0.1) )
classifier.add( Dense(maxlength, activation='softmax') )
'''

classifier = Sequential()
# This will create 32 Feature Maps. Input shape is referring to single sample.
classifier.add(Conv2D(64, (3,3), input_shape=(width, height, 1), activation="relu", padding='same', kernel_initializer='he_normal'))
classifier.add( BatchNormalization() )
classifier.add(Conv2D(64, (3,3), input_shape=(width, height, 1), activation="relu", padding='same', kernel_initializer='he_normal'))
classifier.add( BatchNormalization() )
#classifier.add(MaxPooling2D(pool_size=(2,2), data_format="channels_first")) # (vertical, horizontal)
classifier.add(MaxPooling2D(pool_size=(2,1))) # (vertical, horizontal)
#classifier.add(Dropout(0.1))
# Don't need 'input_shape' because it's already given from previous step.
classifier.add( BatchNormalization() ) #<--------- JUST ADDED AND RAN : F1=0.6944
classifier.add(Conv2D(128, (3,3), activation="relu", padding='same', kernel_initializer='he_normal'))
classifier.add( BatchNormalization() )
classifier.add(Conv2D(128, (3,3), activation="relu", padding='same', kernel_initializer='he_normal'))
classifier.add( BatchNormalization() )
#classifier.add(MaxPooling2D(pool_size=(2,2), data_format="channels_first")) # (vertical, horizontal)
classifier.add(MaxPooling2D(pool_size=(2,1))) # (vertical, horizontal)
#classifier.add(Dropout(0.1))

classifier.add(Flatten())
# Choose units to be not too big but not too small. Common practice is to pick power of 2.
classifier.add(Dense(units=512, activation="relu"))
classifier.add( BatchNormalization() )
#classifier.add( Dropout(0.1) )
classifier.add( Dense(maxlength, activation='softmax') )


In [33]:
learning_rate = 0.1
decay_rate = learning_rate / epochs
momentum = 0.9
sgd = optimizers.SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)

reduce_lr = ReduceLROnPlateau(monitor='val_acc', 
                              patience=3, 
                              verbose=1, 
                              factor=0.5, 
                              min_lr=0.001)

# categorical_crossentropy because we are classifying 2 outcomes.
#classifier.compile(optimizer=optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8), loss="categorical_crossentropy", metrics=['accuracy'])
classifier.compile(optimizer=sgd, loss="categorical_crossentropy", metrics=['accuracy'])

In [34]:
classifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 15, 12, 64)        640       
_________________________________________________________________
batch_normalization_7 (Batch (None, 15, 12, 64)        256       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 15, 12, 64)        36928     
_________________________________________________________________
batch_normalization_8 (Batch (None, 15, 12, 64)        256       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 7, 12, 64)         0         
_________________________________________________________________
batch_normalization_9 (Batch (None, 7, 12, 64)         256       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 7, 12, 128)        73856     
__________

In [35]:
import time

# Fit the model (TRAINING)
X_train = np.array(X_train).reshape(X_train_size, width, height, 1)
X_test = np.array(X_test).reshape(X_test_size, width, height, 1)

start = time.time()
# Trains the model for fixed number of epoches
#classifier.fit(X_train, np.asarray(y_train), callbacks=[reduce_lr], validation_data=(X_test, np.asarray(y_test)), epochs=epochs, batch_size=batch_size)
classifier.fit(X_train, np.asarray(y_train), callbacks=[reduce_lr], validation_split=0.33, epochs=epochs, batch_size=batch_size)
end = time.time()
print("Model took %0.2f seconds" %(end - start))

Train on 8204 samples, validate on 4042 samples
Epoch 1/20
8204/8204 [==============================] - 191s 23ms/step - loss: 16.3972 - acc: 0.1298 - val_loss: 29.3154 - val_acc: 0.0507
Epoch 2/20
8204/8204 [==============================] - 168s 20ms/step - loss: 10.4376 - acc: 0.1927 - val_loss: 12.0944 - val_acc: 0.2004
Epoch 3/20
8204/8204 [==============================] - 184s 22ms/step - loss: 8.8801 - acc: 0.2203 - val_loss: 10.9002 - val_acc: 0.2506
Epoch 4/20
8204/8204 [==============================] - 144s 18ms/step - loss: 8.1231 - acc: 0.2193 - val_loss: 10.4342 - val_acc: 0.2546
Epoch 5/20
8204/8204 [==============================] - 191s 23ms/step - loss: 7.5957 - acc: 0.2303 - val_loss: 10.1461 - val_acc: 0.2199
Epoch 6/20
8204/8204 [==============================] - 193s 24ms/step - loss: 7.2510 - acc: 0.2329 - val_loss: 10.0616 - val_acc: 0.2056
Epoch 7/20
8204/8204 [==============================] - 177s 22ms/step - loss: 7.0767 - acc: 0.2366 - val_loss: 9.9461 - v

In [36]:
classifier.evaluate(X_test, np.asarray(y_test), verbose=1)

6032/6032 [==============================] - 43s 7ms/step


[9.44443052580249, 0.1961206896551724]

In [37]:
prediction = classifier.predict(X_test)

In [38]:
def getF1(sequence_index, label_set, pred, factor):
    
    prob_mean = np.mean(pred[sequence_index])
    prob_std = np.std(pred[sequence_index]) 
    prob_th = prob_mean+factor*prob_std
    
    cur_fp_ls = label_set[sequence_index]
    truePositive = 0
    falseNegative = 0
    for i in range(len(cur_fp_ls)):
        cur_index = cur_fp_ls[i]
        if pred[sequence_index][cur_index] >= prob_th:
            truePositive += 1
        else:
            falseNegative += 1
    
    Positive = 0
    Negative = 0
    for i in range(len(pred[sequence_index])):
        if pred[sequence_index][i] >= prob_th:
            Positive += 1
        else:
            Negative += 1
                
    return truePositive, falseNegative, Positive, Negative

In [40]:
tp_sum = 0
fn_sum = 0
p_sum = 0
n_sum = 0
factor = 2

for i in range(len(rawy_test)):
    tp,fn,p,n = getF1(i, rawy_test.tolist(), prediction,factor)
    tp_sum = tp_sum + tp
    fn_sum = fn_sum + fn
    p_sum = p_sum + p
    n_sum = n_sum + n

precision = tp_sum/p_sum
recall = tp_sum/(tp_sum+fn_sum) 
f1 = 2*precision*recall/(precision+recall)

print("precision = "+str(precision))
print("recall = "+str(recall))
print("f1 = "+str(f1))

precision = 0.518453325638436
recall = 0.83555782914012
f1 = 0.639873214858079


In [ ]:
#TODO :
    1. Use cross-validation to test the model and get the average f1 score.
    2. Graph it out --> using 'each_epoch_end'
    3. Fix the 

In [ ]:
def getF1_new(sequence_index, label_set, pred, factor):
    
    prob_mean = np.mean(pred[sequence_index])
    prob_std = np.std(pred[sequence_index]) 
    prob_th = prob_mean+factor*prob_std
    cur_fp_ls = list(label_set[sequence_index])
    truePositive = 0
    falseNegative = 0
    falsePositive = 0
    for i in range(len(cur_fp_ls)):
        if(cur_fp_ls[i] == 1):
            if pred[sequence_index][i] >= prob_th:
                truePositive += 1
            else:
                falseNegative += 1
        else:
            if pred[sequence_index][i] >= prob_th and cur_fp_ls[i] == 0:
                falsePositive += 1
    
    Positive = 0
    Negative = 0
    for i in range(len(pred[sequence_index])):
        if pred[sequence_index][i] >= prob_th:
            Positive += 1
        else:
            Negative += 1
            
    return truePositive, falseNegative, Positive, Negative

In [ ]:
# For hybrid model only
def random_sample_n_times(X, Y, size):
    X1 = X
    n = X1.shape[0]
    _X1, _Y = [], []
    for i in range(size):
        r = np.random.randint(n)
        _X1.append(X1[r].tolist())
        _Y.append(Y[r].tolist())
        
    _X = np.asarray(_X1)
    _Y = np.asarray(_Y)
    
    return _X, _Y

In [ ]:
from keras.callbacks import Callback
from sklearn.metrics import roc_auc_score, confusion_matrix, f1_score, precision_score, recall_score
import matplotlib.pyplot as plt

class custom_callback(Callback):
    def __init__(self,training_data,validation_data, sample_size):
        self.sample_size = sample_size

        self.x = training_data[0]
        self.y = training_data[1]
        self.x_val = validation_data[0]
        self.y_val = validation_data[1]
        
        self.roc_scores = []
        self.f1_scores = []
        self.precision_scores = []
        self.recall_scores = []
        

    def on_train_begin(self, logs={}):
        self.roc_scores = []
        self.f1_scores = []
        self.precision_scores = []
        self.recall_scores = []
        return
    
    def on_epoch_end(self, epoch, logs={}):
        x, y = random_sample_n_times(self.x, self.y, self.sample_size)
        x_val, y_val = random_sample_n_times(self.x_val, self.y_val, self.sample_size)

        y_pred = self.model.predict(x)
        y_pred_val = self.model.predict(x_val)
    
        tp_sum = 0
        fn_sum = 0
        p_sum = 0
        n_sum = 0
        
        # calculate f1 score in the batch from validation set
        for i in range(self.sample_size):
            tp,fn,p,n = getF1_new(i, y_val, y_pred_val,4)
            tp_sum = tp_sum + tp
            fn_sum = fn_sum + fn
            p_sum = p_sum + p
            n_sum = n_sum + n

        precision = tp_sum/p_sum
        recall = tp_sum/(tp_sum+fn_sum) 
        f1 = 2*precision*recall/(precision+recall)

        self.f1_scores.append((len(self.f1_scores),f1))
        
        return

In [ ]:
X_train = np.array(X_train).reshape(X_train_size, width, height, 1)
X_test = np.array(X_test).reshape(X_test_size, width, height, 1)

cb = custom_callback((X_train, np.asarray(y_train)), (X_test, np.asarray(y_test)), 128)

start = time.time()
# Trains the model for fixed number of epoches
#classifier.fit(X_train, np.asarray(y_train), callbacks=[cb], validation_data=(X_test, np.asarray(y_test)), epochs=epochs, batch_size=batch_size)
classifier.fit(X_train, np.asarray(y_train), callbacks=[cb], validation_split=0.33, epochs=epochs, batch_size=batch_size)
end = time.time()

In [ ]:
print(cb.f1_scores)

In [ ]:
fig1 = plt.figure()
ax1 = fig1.add_subplot(111)
ax1.scatter(*zip(*cb.f1_scores))
ax1.set_ylabel('f1')
ax1.set_xlabel('epoch')

In [ ]:
f1_scores = 0
for score in cb.f1_scores : 
    f1_scores += score[1]

print('Average f1 score : ', f1_scores/len(cb.f1_scores))